In [1]:
# Import packages.
from petsc4py import PETSc
from slepc4py import SLEPc

import numpy as np
import scipy

from matplotlib import pyplot as plt
import csv
import sys

# Check anaconda environment. Script needs complex petsc/slepc. Scalar type should be complex128.
from petsc4py import PETSc
print("Scalar type: " + str(PETSc.ScalarType))

Scalar type: <class 'numpy.complex128'>


In [2]:
Re = 200
base_dir = './backward_facing_step_compressible/'
case_path = base_dir + str(Re) + '/'

In [3]:
# Load matrices.
L = scipy.sparse.load_npz(case_path + 'L.npz')
A = scipy.sparse.load_npz(case_path + 'A.npz')

# Important: Here equations are formulatet as (\sigma A + L)q = 0.
# However, slepc expects Lq = \sigma A q.
# => Multiply A matrix with -1, before stability analysis.
A = A * (-1)

In [4]:
# # Visualize the sparsity structure using spy plots.
# plt.figure(figsize=(10, 4))
# plt.subplot(1, 2, 1)
# plt.title("Sparsity pattern of A")
# plt.spy(A, markersize=2)

# plt.subplot(1, 2, 2)
# plt.title("Sparsity pattern of L")
# plt.spy(L, markersize=2)
# plt.tight_layout()
# plt.show()

In [5]:
# Convert from scipy to petsc matrices.
L_pet = PETSc.Mat().createAIJ(size=L.shape, csr=(L.indptr, L.indices, L.data))
A_pet = PETSc.Mat().createAIJ(size=A.shape, csr=(A.indptr, A.indices, A.data))

A_pet_t = A_pet.copy()
A_pet_t = A_pet_t.transpose()

In [6]:
def normalize_matrix(M):
    """
    In-place normalization of a PETSc AIJ matrix M column-by-column using its CSR data.
    
    For each column j, compute:
      mean[j] = (sum_i M(i,j)) / m
      std[j]  = sqrt( (sum_i |M(i,j)|^2)/m - |mean[j]|^2 )
      
    Then update each stored nonzero entry:
      M(i,j) = (M(i,j) - mean[j]) / std[j]
      
    The matrix M is modified in-place.
    """
    # Get global sizes as integers.
    m_global, n_global = M.getSize()
    m_global = int(m_global)
    n_global = int(n_global)

    ia, ja, a = M.getValuesCSR()
    ja = ja.ravel()  # ensure column indices are 1D

    # Allocate per-column accumulators.
    col_sum   = np.zeros(n_global, dtype=a.dtype)
    col_sumsq = np.zeros(n_global, dtype=np.float64)

    # Accumulate sums from the stored nonzero entries.
    np.add.at(col_sum, ja, a)
    np.add.at(col_sumsq, ja, np.abs(a)**2)
    
    # Compute per-column mean and standard deviation.
    mean = col_sum / m_global
    std  = np.sqrt(col_sumsq / m_global - np.abs(mean)**2)
    
    # Update the nonzero entries in-place.
    a[:] = (a - mean[ja]) / std[ja]
    
    # Commit the new values back to M.
    M.setValuesCSR(ia, ja, a)
    M.assemble()
    return mean, std

# Normalize matrices A and L.

meanA, stdA = normalize_matrix(A_pet_t)
meanL, stdL = normalize_matrix(L_pet)

A_pet.destroy()
A_pet = A_pet_t
A_pet = A_pet.transpose()


In [7]:
A_H_pet = A_pet.copy()
A_H_pet.hermitianTranspose()

L_H_pet = L_pet.copy()
L_H_pet.hermitianTranspose()

In [8]:
n_ev = 30

# Setup eigenvalue problem.
LEP = SLEPc.EPS().create()
LEP.setOperators(L_pet*L_H_pet, A_pet*A_H_pet)
LEP.setProblemType(SLEPc.EPS.ProblemType.GNHEP)
LEP.setType(SLEPc.EPS.Type.KRYLOVSCHUR)
LEP.setDimensions(n_ev, PETSc.DECIDE, PETSc.DECIDE)
# LEP.setTwoSided(True)
LEP.setWhichEigenpairs(SLEPc.EPS.Which.LARGEST_REAL)
# LEP.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_MAGNITUDE)
LEP.setTolerances(1e-9, 5000)

st = LEP.getST()
ksp = st.getKSP()

ksp.setType("preonly")
pc = ksp.getPC()
pc.setType("jacobi")

LEP.setFromOptions()
LEP.view()

# Solve eigenvalue problem.
LEP.solve()

# Access solution.
tol, maxit = LEP.getTolerances()
nconv      = LEP.getConverged()
    
print("Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit))
print("Number of converged eigenpairs %d" % nconv)

EPS Object: 1 MPI process
  type: krylovschur
    0% of basis vectors kept after restart
    using the locking variant
  problem type: generalized non-hermitian eigenvalue problem
  selected portion of the spectrum: largest real parts
  postprocessing eigenvectors with purification
  number of eigenvalues (nev): 30
  number of column vectors (ncv): -1
  maximum dimension of projected problem (mpd): -1
  maximum number of iterations: 5000
  tolerance: 1e-09
  convergence test: relative to the eigenvalue
BV Object: 1 MPI process
  type: mat
  0 columns of global length -1
  vector orthogonalization method: classical Gram-Schmidt
  orthogonalization refinement: if needed (eta: 0.7071)
  block orthogonalization method: GS
  doing matmult as a single matrix-matrix product
DS Object: 1 MPI process
  type: nhep
ST Object: 1 MPI process
  type: shift
  shift: 0.
  number of matrices: 2
  nonzero pattern of the matrices: UNKNOWN
  KSP Object: (st_) 1 MPI process
    type: preonly
    maximum it

In [9]:
spectrum = np.zeros(nconv, dtype=complex)

# Create vectors of dimensions of matrix.
evR = L_pet.getVecs()[0].duplicate()
evR_array = []

print()
print("        k          ||Ax-kx||/||kx|| ")
print("----------------- ------------------")

for iEv in range(nconv):

    k     = LEP.getEigenvalue(iEv)
    error = LEP.getErrorEstimate(iEv)
        
    LEP.getEigenvector(iEv, evR)

    spectrum[iEv] = k

    if k.imag != 0.0:
        print(" %9g%+9g j %12g" % (k.real, k.imag, error))
    else:
        print(" %12f      %12g" % (k.real, error))

    evR_array.append(evR.copy())


        k          ||Ax-kx||/||kx|| 
----------------- ------------------
   6.38406+2.01425e-16 j  8.10934e-11
   6.38406+1.74441e-17 j  2.47592e-10
   6.38146+4.99718e-15 j  6.99555e-11
   6.38146+1.03888e-15 j  4.08168e-10
   6.38074+2.10307e-15 j   7.0136e-14
   6.38071-1.18967e-15 j  1.64319e-14
   6.37886+2.39862e-15 j  1.98375e-10
   6.37886-1.19159e-16 j  7.29419e-10
   6.37625+1.44226e-15 j  5.06437e-11
   6.37625+2.75757e-15 j   1.3027e-10
   6.37363+8.0393e-15 j  5.67053e-10
   6.37363-2.1799e-15 j  1.44703e-10
     6.371-4.04088e-15 j  7.17036e-10
     6.371+6.16091e-15 j  7.64415e-10
   6.36833-5.09118e-15 j  8.09817e-10
   6.36833-5.42111e-16 j  5.25318e-10
   6.36706-6.55441e-15 j   2.7613e-10
   6.36706+4.54466e-15 j  4.16565e-11
   6.36647-9.26639e-16 j  3.99185e-10
   6.36646-1.46113e-15 j  4.21927e-10
   6.36566+1.41688e-15 j  5.37633e-10
   6.36565+5.71384e-15 j  6.82793e-10
   6.36557+1.82835e-15 j  3.12278e-10
   6.36556-6.79486e-17 j  9.33981e-11
   6.36441-2.25

In [10]:
svdU = L_pet.getVecs()[0].duplicate()

svdU_array = []

for iEv in range(nconv):
    
    A_H_pet.mult(evR_array[iEv], svdU)
    # normalize eigenvector
    # norm_val = svdU.norm(PETSc.NormType.NORM_2)
    # svdU.scale(1.0 / norm_val)
    svdU_array.append(svdU.copy())

In [11]:
def gram_schmidt(vec_list):
    """Perform modified Gram-Schmidt on a list of PETSc Vec objects."""
    orthonormal_vecs = []
    for v in vec_list:
        # Create a new copy to avoid modifying the original vector.
        v_copy = v.copy()
        for u in orthonormal_vecs:
            # Compute the projection coefficient alpha.
            alpha = v_copy.dot(u)
            # Remove the component in the direction u.
            v_copy.axpy(-alpha, u)
        # Normalize the vector.
        norm = v_copy.norm(PETSc.NormType.NORM_2)
        v_copy.scale(1.0 / norm)
        orthonormal_vecs.append(v_copy)
    return orthonormal_vecs

svdU_array = gram_schmidt(svdU_array)

In [12]:
#check for orthogonality
max_val = 0
for i in range(len(svdU_array)):
    for j in range(i+1, len(svdU_array)):
        val = svdU_array[i].dot(svdU_array[j])
        print("non-orthogonality: ", np.abs(val))
        max_val = max(max_val, np.abs(val.real))
print("Max non-orthogonality: ", max_val)

non-orthogonality:  1.1641770465836412e-14
non-orthogonality:  5.53472127047136e-16
non-orthogonality:  1.6223083480138551e-15
non-orthogonality:  2.7464294326054307e-24
non-orthogonality:  3.1938273057140058e-24
non-orthogonality:  5.790862671364025e-17
non-orthogonality:  2.0596770883286915e-17
non-orthogonality:  3.968411203596786e-18
non-orthogonality:  1.5304740269913437e-17
non-orthogonality:  4.928543212114626e-18
non-orthogonality:  2.5680626188272433e-18
non-orthogonality:  7.28152788518769e-18
non-orthogonality:  2.9800712790940413e-18
non-orthogonality:  8.351545257958314e-18
non-orthogonality:  5.245850960727642e-18
non-orthogonality:  8.285292080956808e-20
non-orthogonality:  4.304523972407485e-20
non-orthogonality:  3.862822540613761e-20
non-orthogonality:  1.8679599042585107e-19
non-orthogonality:  1.1655935502217682e-18
non-orthogonality:  1.142302212882913e-18
non-orthogonality:  3.462374158904566e-18
non-orthogonality:  4.864377454936049e-18
non-orthogonality:  3.0384

In [13]:
#diagonal singular value matrix sigma
sigma = np.sqrt(spectrum)

In [14]:
# Create a KSP solver for L_pet.
ksp = PETSc.KSP().create()
ksp.setOperators(L_pet)
ksp.setType("preonly")
pc = ksp.getPC()
pc.setType("lu")
ksp.setFromOptions()

# Create PETSc vectors for solution and right-hand-side.
svdV = L_pet.getVecs()[0].duplicate()
rhs = A_pet.getVecs()[0].duplicate()

svdV_array = []

# Solve for the i-th vector v[i].
for i in range(len(svdU_array)):

    # Compute rhs = sigma[i] * (A * svdU_array[i])
    A_pet.mult(svdU_array[i], rhs)
    rhs.scale(sigma[i])  # sigma stored as diagonal matrix

    # Solve L * v = rhs
    ksp.solve(rhs, svdV)
    #check for divergence
    if np.isnan(1 / svdV.getArray()[0]):
        break

    svdV_array.append(svdV.copy())

In [15]:
#check for orthogonality
max_val = 0
for i in range(len(svdV_array)):
    for j in range(i+1, len(svdV_array)):
        val = svdV_array[i].dot(svdV_array[j])
        print("non-orthogonality: ", np.abs(val))
        max_val = max(max_val, np.abs(val.real))
print("Max non-orthogonality: ", max_val)

non-orthogonality:  4.647994857351434e-08
non-orthogonality:  3.926483129999183
non-orthogonality:  1.1722425899355207
non-orthogonality:  8.124985068696242e-09
non-orthogonality:  3.980235162873288e-08
non-orthogonality:  0.5918566725132078
non-orthogonality:  0.06980121063347591
non-orthogonality:  0.0383627656897446
non-orthogonality:  0.017869491067929077
non-orthogonality:  0.002340460067824422
non-orthogonality:  0.0031665637822059925
non-orthogonality:  7.147988255222715e-05
non-orthogonality:  0.00012537442196594868
non-orthogonality:  3.4624995573397694e-05
non-orthogonality:  3.1898331663396534e-05
non-orthogonality:  5.144880368992336e-06
non-orthogonality:  3.834985009666925e-06
non-orthogonality:  7.630519316071465e-06
non-orthogonality:  5.706815761947725e-06
non-orthogonality:  2.401352405108425e-05
non-orthogonality:  1.373807512273817e-05
non-orthogonality:  5.847284934844825e-05
non-orthogonality:  2.6181708700614614e-05
non-orthogonality:  2.2620725064141777e-05
non-

# export SVD

In [16]:
import os
os.makedirs(case_path + 'svd/', exist_ok=True)

In [17]:
for iEv in range(nconv):
    np.savez(case_path + 'svd/' + str(iEv), svdU=svdU_array[iEv].getArray(), svdV=svdV_array[iEv].getArray())

In [18]:
# for i in range(len(EWs)):
#     sv_data = np.load(case_path + 'svd/' + str(i) + '.npz')
#     svdU_np = sv_data['svdU']
#     svdV_np = sv_data['svdV']

#     equation.q_real_list[equation.dof["u"]].vector().set_local(np.real(svdV_np[equation.VMixed.sub(equation.dof["u"]).dofmap().dofs()]))
#     equation.q_imag_list[equation.dof["u"]].vector().set_local(np.imag(svdV_np[equation.VMixed.sub(equation.dof["u"]).dofmap().dofs()]))
#     equation.f_real_list[equation.dof["u"]].vector().set_local(np.real(svdU_np[equation.VMixed.sub(equation.dof["u"]).dofmap().dofs()]))
#     equation.f_imag_list[equation.dof["u"]].vector().set_local(np.imag(svdU_np[equation.VMixed.sub(equation.dof["u"]).dofmap().dofs()]))

#     fields_to_write["svdU_real"] = equation.f_real_list[equation.dof["u"]]
#     fields_to_write["svdU_imag"] = equation.f_imag_list[equation.dof["u"]]
#     fields_to_write["svdV_real"] = equation.q_real_list[equation.dof["u"]]
#     fields_to_write["svdV_imag"] = equation.q_imag_list[eation.dof["u"]]

#     # Write eigenmodes and singular vectors.
#     io = Io()
#     io.write_paraview(geometry, settings, "SVD_" + str(i), fields_to_write)

In [19]:
!jupyter nbconvert --to script svd_test.ipynb

[NbConvertApp] Converting notebook svd_test.ipynb to script
[NbConvertApp] Writing 8208 bytes to svd_test.py
